In [102]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [104]:
import pandas as pd 
import numpy as np
from preprocessing.cleaning_utils import *
from functools import partial
import torch 

from datasets import load_dataset
from datasets import Value, ClassLabel, Features, DatasetDict
import transformers
from transformers import AutoTokenizer, AutoModel, AutoModelForMaskedLM, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import logging
from transformers import TrainingArguments, Trainer
from transformers import pipeline
import spacy
import spacy
from spacy import displacy
from spacy.symbols import nsubj, VERB, nmod, ccomp, punct, conj
from spacy.tokens import Doc, Span, Token
logging.set_verbosity_warning()

from preprocessing.cleaning_utils import *

In [3]:
# !python -m torch.utils.collect_env

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [5]:
mimic_dir = "/home/vs428/project/MIMIC/files/mimiciii/1.4/"
n2c2_dir = "/home/vs428/project/n2c2/2022/N2C2-AP-Reasoning/"
n2c2_data_dir =  "/home/vs428/project/n2c2/2022/Data/"


In [111]:
classes = ['Not Relevant', 'Neither', 'Indirect', 'Direct']
features = Features({
    'ROW ID':Value("int64"),
    'HADM ID':Value("int64"),
    'Assessment':Value("string"),
    'Plan Subsection':Value("string"),
    "Relation":Value("string")
}) 

dataset = load_dataset("parquet", data_files={
                            "train":n2c2_data_dir + "train-abbv.parquet",
                            "valid":n2c2_data_dir + "valid-abbv.parquet",
                        },

                       # features=features
                      )
# dataset = dataset.class_encode_column("Relation")
# dataset = dataset.rename_column("Relation", "label")

Using custom data configuration default-346727d25b56a84a
Reusing dataset parquet (/home/vs428/.cache/huggingface/datasets/parquet/default-346727d25b56a84a/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


  0%|          | 0/2 [00:00<?, ?it/s]

In [112]:
dataset = dataset.map(remove_MIMIC_deid)

Loading cached processed dataset at /home/vs428/.cache/huggingface/datasets/parquet/default-346727d25b56a84a/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901/cache-b8a567059a696c67.arrow
Loading cached processed dataset at /home/vs428/.cache/huggingface/datasets/parquet/default-346727d25b56a84a/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901/cache-ce5e410dc73c0574.arrow


In [110]:
dataset['train'][125]

{'ROW ID': 560236,
 'HADM ID': 199097,
 'Assessment': 'Patient is a 58 year old male with history of atrial fibrillation on\n   coumadin, hyperlipidemia, hypertension, peripheral vascular disease,\n   status post costovertebral angle who presented to [**Hospital3 847**] on [**2185-3-2**] with heart\n   failure exacerbation and atrial fibrillation with rapid ventricular response , transferred due\n   to difficulty with diuresis [**2-7**] hypotension.',
 'Plan Subsection': '#. fluids , electrolytes , nutrition \n   - Low skin associated lymphoid tissue , cardiac heart healthy diet\n   - Monitor electrolytes and replete pro re nata \n   .\n   #. Access: PIVs\n   .\n   #. prophylaxis :\n   - proton pump inhibitor [**Hospital1 **], bowel regimen, supratherapeutic international normalized ratio \n   .\n   #. Code:  Full\n   .\n   #. disposition :  coronary care unit for now.\n   - posterior tibialis consult in the AM.\n   intensive care unit Care\n   Nutrition:\n   Glycemic Control:\n   Line

In [8]:
from transformers import GPT2Tokenizer, GPTNeoForSequenceClassification


In [9]:
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})


1

In [11]:
model = GPTNeoForSequenceClassification.from_pretrained("EleutherAI/gpt-neo-1.3B", num_labels=4, 
                                                        problem_type="single_label_classification",
                                                       pad_token_id=tokenizer.convert_tokens_to_ids("[PAD]"))



Some weights of GPTNeoForSequenceClassification were not initialized from the model checkpoint at EleutherAI/gpt-neo-1.3B and are newly initialized: ['transformer.h.21.attn.attention.bias', 'transformer.h.9.attn.attention.bias', 'transformer.h.17.attn.attention.bias', 'transformer.h.5.attn.attention.bias', 'score.weight', 'transformer.h.11.attn.attention.bias', 'transformer.h.13.attn.attention.bias', 'transformer.h.19.attn.attention.bias', 'transformer.h.23.attn.attention.bias', 'transformer.h.7.attn.attention.bias', 'transformer.h.15.attn.attention.bias', 'transformer.h.1.attn.attention.bias', 'transformer.h.3.attn.attention.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [112]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50258, 2048)

In [113]:
out = tokenizer(dataset['train'][43:45]['Assessment'], dataset['train'][43:45]['Plan Subsection'],
                                      truncation="longest_first",
                                    # padding=True,
                                      max_length=512,
                                      verbose=True)

In [125]:
generator = pipeline(task="text-classification", model=model, tokenizer=tokenizer, return_all_scores=True)

In [128]:
generator([dataset['train'][43]['Assessment'] +  dataset['train'][43]['Plan Subsection'], 
           dataset['train'][44]['Assessment'] +  dataset['train'][44]['Plan Subsection']])[0]

[{'label': 'LABEL_0', 'score': 0.45915326476097107},
 {'label': 'LABEL_1', 'score': 0.03891521692276001},
 {'label': 'LABEL_2', 'score': 0.478484570980072},
 {'label': 'LABEL_3', 'score': 0.02344704233109951}]

In [134]:
x = np.zeros(shape=(5,5))

In [138]:
x[4, :] = [1,2,3,4,5 ]

In [139]:
x

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [1., 2., 3., 4., 5.]])

In [136]:
data_collator = DataCollatorWithPadding(tokenizer,
                                        max_length=512, 
                                        padding="max_length",
                                        return_tensors="pt")    


In [116]:
test = data_collator(out)

In [117]:
# tokenizer.decode(out['input_ids'])
model_out = model(**test)

In [118]:
model_out.logits

tensor([[ 1.5115, -0.9565,  1.5527, -1.4632],
        [ 0.1700, -2.3643,  3.2297, -0.9741]], grad_fn=<IndexBackward0>)

In [ ]:
dataset = dataset.map(partial(tokenize_function, tokenizer=tokenizer), batched=True)

# tokenizer(data['train'][23])

In [8]:
# # split dataset: 80% train, 10% validation, 10% test 
# train_testvalid = dataset['train'].train_test_split(test_size=0.2)
# # test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
# split_dataset = DatasetDict({
#     'train': train_testvalid['train'],
#     'valid': test_valid['train'],
#     # 'test': test_valid['test'],
# })



In [113]:
# prompts = []
# for data in dataset['train']:
#     prompt_str = 'Assessment: "' + data['Assessment'] + '"\nPlan: "' + data['Plan Subsection'] + '"\nLabel: ' \
#         + str(data['label'])
#     prompts.append(prompt_str)
prompts = []
for data in dataset['train']:
    prompt_str = 'Assessment: ' + data['Assessment'] + '\nPlan: ' + data['Plan Subsection'] + '\nLabel: ' \
        + str(data['label'])
    prompts.append(prompt_str)
        

In [114]:
prompts[10]

'Assessment: Assesment: This is a 76 year-old female with a history of diabetes who\n   presents with pneumoperitoneum of unclear aetiology, complicated by\n   coagulopathy and rhabdomyolysis.\nPlan: # Acute renal failure -  be pre-renal or secondary to\n   rhabdomyolysis. Creatinine down from 1.7 to 1.3.\n   - Continue aggressive 4 fluid hydration\n   - Monitor potassium (3.8 this AM)\n   - Renally dose antibiotics\nLabel: Indirect'

In [115]:
for idx, x in enumerate(prompts):
    print(x)
    print("\n")
    if idx % 5 == 0 and idx > 5:
        break

Assessment: 51 year old F with a history of 3V coronary artery disease , confirmed on C. catheter during this
   admission, ejection fraction of 40percent , who is transferred to coronary care unit for monitoring of
   recurrent chest pain. Plan for coronary artery bypass graft today to revascularize due to
   3-vessel disease.
Plan: # CORONARIES: Patient with 3 vessel disease on catheter . Previously found
   to be the case in , but had been treated medically. On the floor,
   patient with recurrent chest pain and electrocardiogram changes, concerning for
   further ischemia. Chest pain has been controlled on nitro drop . Patient
   in coronary care unit for monitoring; plan for coronary artery bypass graft today.
   - continue nitro drop 
   - if has further chest pain , would uptitrate nitro, and contact  and
   attending for potential balloon pump versus urgent coronary artery bypass graft 
   - appreciate cardiothoracic  recommendations 
   - continue , , ACE-i, simvastatin
Label:

In [116]:
arr = np.array(prompts) 

In [117]:
idxs = np.random.randint(arr.shape[0], size=5)
for x in idxs:
    print(x)
    print(arr[x])
    print("\n-------------------\n")

1720
Assessment: active/passive : 56 year old M with history of hemochromatosis and history of community acquired pneumonia ,
   presents with left lower lid pneumonia and hypoxia.
Plan: # Acute Renal Failure - creatinine 1.7 and special  of >1.035, likely prerenal
   azotemia.  Baseline 1.0 in .
   - continue intravenous fluid overnight, recheck BUN/Cr in AM
Label: Neither

-------------------

1116
Assessment: 68 year old M with severe NYHA class 4 systolic congestive heart failure with ejection fraction of 20percent ,
   secondary to ischemic cardiomyopathy, admitted  for acute
   exacerbation of congestive heart failure and transferred to coronary care unit from  service for
   aggressive diuresis and possible RRT for acute on chronic renal
   failure.
Plan: # CORONARIES: coronary artery disease status post myocardial infarction in , coronary artery bypass graft xanthosine 2. Cardiac catheter showed
   right-dominant system with 50percent mid-RCA stenosis, patent SVG-D1 and
   SVG-

# SciSpacy Dependency Parsing

In [20]:
idxs = np.random.randint(arr.shape[0], size=5)


In [21]:
from scispacy.abbreviation import AbbreviationDetector
Token.set_extension("is_beginning_capitalized", default=False)

In [22]:
# nlp = spacy.load("en_core_sci_scibert")
nlp = spacy.load("en_core_sci_lg")
nlp.add_pipe("abbreviation_detector")

In [118]:
doc = nlp("Spinal and bulbar muscular atrophy (SBMA) is an \
           inherited motor neuron disease caused by the expansion \
           of a polyglutamine tract within the androgen receptor (AR). \
           SBMA can be caused by this easily.")

print("Abbreviation", "\t", "Definition")
for abrv in doc._.abbreviations:
	print(f"{abrv} \t ({abrv.start}, {abrv.end}) {abrv._.long_form}")

Abbreviation 	 Definition
SBMA 	 (33, 34) Spinal and bulbar muscular atrophy
SBMA 	 (6, 7) Spinal and bulbar muscular atrophy
AR 	 (29, 30) androgen receptor


In [119]:
# docs = ["35 yo F with SLE, restrictive lung disease, CM (EF 15-20%) presents with severe lupus induced cardiomyopathy.",
#         "54 year old man with pmh significant for bipolar disorder presenting with lithium toxicity",
#         "69 F w/ MMP including copd, diastolic chf, dm2, recent prolonged hospitalization where she became trach- dependent secondary to prolonged virulent pseudomonal PNA, presented in septic shock, ARF and AMS.",
#         "73 yo male with history of bilateral renal cell carcinoma metastatic to right adrenalectomy presents for post-operative monitoring after right adrenalectomy and prostate biopsy."]

assessments = [data['Assessment'] for data in dataset['train']]
assessments = np.array(assessments)
assessments[idxs]


array(['active/passive : 56 year old M with history of hemochromatosis and history of community acquired pneumonia ,\n   presents with left lower lid pneumonia and hypoxia.',
       '68 year old M with severe NYHA class 4 systolic congestive heart failure with ejection fraction of 20percent ,\n   secondary to ischemic cardiomyopathy, admitted  for acute\n   exacerbation of congestive heart failure and transferred to coronary care unit from  service for\n   aggressive diuresis and possible RRT for acute on chronic renal\n   failure.',
       '~30 yoM with unknown PMH/social history , admitted after being found\n   unresponsive while breaking into elementary school.',
       '87yo woman with history of coronary artery disease , atrial fibrillation on dofetalide, history of vessel tach status post PPM/ICD,\n   CML on Gleevec but not taking, type 2 diabetes mellitus, chronic kidney disease , who\n   initially presented to the bone marrow transplant service with vertigo and vomiting on\n   

In [120]:
import spacy

doc = nlp("Credit and mortgage account holders must submit their requests")
# span = doc[doc[4].left_edge.i : doc[4].right_edge.i+1]
# with doc.retokenize() as retokenizer:
#     retokenizer.merge(span)
# for token in doc:
#     print(token.text, token.pos_, token.dep_, token.head.text)

/gpfs/ysm/project/rtaylor/vs428/conda_envs/myenv/lib/python3.10/site-packages/scispacy/abbreviation.py:230: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)


In [121]:
processed_docs = []

for idx in idxs:
    doc_string = assessments[idx]
    doc = nlp(str(doc_string))
    left_most_edge = len(doc)
    right_most_edge = 0
    beginning_prob_list = []
    continuing_upper = False
    
    print(f"SENTENCE: {idx}")
    print(doc)
    
    start_token = doc[0]
    if start_token.is_upper and start_token.is_alpha:
        continuing_upper = True  
        start_token._.is_beginning_capitalized = True
    else:
        start_token._.is_beginning_capitalized = False
    
    for token_idx, possible_subject in enumerate(doc):
        # extract all the capitalized words at the beginning        
        if continuing_upper and (possible_subject.is_upper or possible_subject.is_punct or possible_subject.is_space):
            beginning_prob_list.append(possible_subject)
            possible_subject._.is_beginning_capitalized = True
            continue
        continuing_upper = False
            
    
    processed_docs.append(doc)
    print("SDFSDFSDF", [token.text for token in doc if not token._.is_beginning_capitalized])
    # processed_docs.append([token.text for token in doc if not token._.is_beginning_capitalized])
    print("\n---------")
    print("---------")    


SENTENCE: 1720
active/passive : 56 year old M with history of hemochromatosis and history of community acquired pneumonia ,
   presents with left lower lid pneumonia and hypoxia.
SDFSDFSDF ['active/passive', ':', '56', 'year', 'old', 'M', 'with', 'history', 'of', 'hemochromatosis', 'and', 'history', 'of', 'community', 'acquired', 'pneumonia', ',', '\n   ', 'presents', 'with', 'left', 'lower', 'lid', 'pneumonia', 'and', 'hypoxia', '.']

---------
---------
SENTENCE: 1116
68 year old M with severe NYHA class 4 systolic congestive heart failure with ejection fraction of 20percent ,
   secondary to ischemic cardiomyopathy, admitted  for acute
   exacerbation of congestive heart failure and transferred to coronary care unit from  service for
   aggressive diuresis and possible RRT for acute on chronic renal
   failure.
SDFSDFSDF ['68', 'year', 'old', 'M', 'with', 'severe', 'NYHA', 'class', '4', 'systolic', 'congestive', 'heart', 'failure', 'with', 'ejection', 'fraction', 'of', '20percent', 

In [122]:
print(processed_docs)

[active/passive : 56 year old M with history of hemochromatosis and history of community acquired pneumonia ,
   presents with left lower lid pneumonia and hypoxia., 68 year old M with severe NYHA class 4 systolic congestive heart failure with ejection fraction of 20percent ,
   secondary to ischemic cardiomyopathy, admitted  for acute
   exacerbation of congestive heart failure and transferred to coronary care unit from  service for
   aggressive diuresis and possible RRT for acute on chronic renal
   failure., ~30 yoM with unknown PMH/social history , admitted after being found
   unresponsive while breaking into elementary school., 87yo woman with history of coronary artery disease , atrial fibrillation on dofetalide, history of vessel tach status post PPM/ICD,
   CML on Gleevec but not taking, type 2 diabetes mellitus, chronic kidney disease , who
   initially presented to the bone marrow transplant service with vertigo and vomiting on
   , and a white count of 173 thought to be du

In [123]:
# processed_docs = []
N =  2

# for doc in processed_docs:
doc = processed_docs[N]
left_most_edge = len(doc)
right_most_edge = 0

print(f"SENTENCE: {idx}")
print(doc)

for token_idx, token in enumerate(doc):
    # if possible_subject.dep in [nmod, ccomp, punct, conj] and 
    # possible_subject.head.text in ["presents","presenting","presented", "admitted", "transferred"]:
    if token.pos_ in ['NOUN', "ADJ"]:
        print(token)
        print(token.head)
        if token.head.lemma_ in ["here", "present", "admit", "transfer", "monitor", "transfer", "p/w"]:
            print(token)
    # if possible_subject.dep in [nmod, ccomp, punct, conj] and (possible_subject.lemma_ in ["here", "present", "admit", "transfer", "monitor", "transfer", "p/w"]):
        # for descendant in possible_subject.subtree:
        #     print(descendant)
        # print("possible_subject", possible_subject)
        # print("span", list(possible_subject.subtree))
        print("--\n")


SENTENCE: 3944
~30 yoM with unknown PMH/social history , admitted after being found
   unresponsive while breaking into elementary school.
yoM
yoM
--

unknown
history
--

PMH/social
history
--

history
yoM
--

unresponsive
found
--

elementary
school
--

school
breaking
--



In [124]:
print(processed_docs[N])
excluding_caps_docs = []
for doc in processed_docs:
    excluding_caps = [(token, token.whitespace_) for token in doc if ((not token._.is_beginning_capitalized) and (not token.is_space))]
    excluding_caps_docs.append(excluding_caps)
# print(processed_docs[0][excluding_caps])
# for sent in processed_docs[0].sents:
#     print(sent)

~30 yoM with unknown PMH/social history , admitted after being found
   unresponsive while breaking into elementary school.


In [125]:
# # vals = [processed_docs[N][idx].text for idx in range(len(processed_docs[N])) if idx in excluding_caps]
# " ".join([x.text for x in excluding_caps_docs[0]])
excluded_caps_text = []
for doc in excluding_caps_docs:
    doc_text = ""
    for tok, whitespace in doc:
        doc_text += tok.text + whitespace
    excluded_caps_text.append(doc_text)

In [126]:
excluded_caps_text

['active/passive : 56 year old M with history of hemochromatosis and history of community acquired pneumonia ,presents with left lower lid pneumonia and hypoxia.',
 '68 year old M with severe NYHA class 4 systolic congestive heart failure with ejection fraction of 20percent ,secondary to ischemic cardiomyopathy, admitted for acuteexacerbation of congestive heart failure and transferred to coronary care unit from service foraggressive diuresis and possible RRT for acute on chronic renalfailure.',
 '~30 yoM with unknown PMH/social history , admitted after being foundunresponsive while breaking into elementary school.',
 '87yo woman with history of coronary artery disease , atrial fibrillation on dofetalide, history of vessel tach status post PPM/ICD,CML on Gleevec but not taking, type 2 diabetes mellitus, chronic kidney disease , whoinitially presented to the bone marrow transplant service with vertigo and vomiting on, and a white count of 173 thought to be due to untreated CML.Now trans

In [133]:
from pathlib import Path

In [144]:
svg = displacy.render(nlp(excluded_caps_text[0]), style="dep", 
                jupyter=False,
                options={
    # "collapse_phrases":True, 
    "distance":75
})
output_path = Path("./images/dep_plot1.svg") 
output_path.open("w", encoding="utf-8").write(svg)

18889

In [140]:
svg = displacy.render(nlp(excluded_caps_text[1]), style="dep", 
                jupyter=False,
                options={
    # "collapse_phrases":True, 
    "distance":85
})
output_path = Path("./images/dep_plot2.svg") 
output_path.open("w", encoding="utf-8").write(svg)

39065

In [142]:
svg = displacy.render(nlp(excluded_caps_text[2]), style="dep", 
                jupyter=False,
                options={
    # "collapse_phrases":True, 
    "distance":75
})
# output_path = Path("./images/dep_plot3.svg") 
# output_path.open("w", encoding="utf-8").write(svg)

In [34]:
# for chunk in doc.noun_chunks:
#     print(chunk.text, chunk.root.text, chunk.root.dep_,
#                 chunk.root.head.text)    

In [ ]:
for token in processed_docs[1]:
    if token.text == "presenting":
        for descendant in token.subtree:
            print(descendant)
    #     assert subject is descendant or subject.is_ancestor(descendant)
    #     print(descendant.text, descendant.dep_, descendant.n_lefts,
    #             descendant.n_rights,)


In [ ]:
for doc_string in docs:
    doc = nlp(doc_string)
    processed_docs.append(doc)
    # print("Abbreviations:", doc._.abbreviations)
    # for abrv in doc._.abbreviations:
    #     print(f"{abrv} \t ({abrv.start}, {abrv.end}) {abrv._.long_form}")    
    for token in doc:
        if token.text in ["presents","presenting","presented", "admitted"]:
            print([subtoken.text for subtoken in token.lefts])  # ['bright', 'red']
            print([subtoken.text for subtoken in token.rights])  # ['on']



In [ ]:
bio_clinicalbert_tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

In [ ]:
bio_clinicalbert_model = AutoModelForSequenceClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", num_labels=4)

In [ ]:
training_args = TrainingArguments(output_dir="test_trainer", 
                                  evaluation_strategy="epoch",
                                  
)

In [ ]:
import numpy as np
from datasets import load_metric

acc = load_metric("accuracy")
macrof1 = load_metric("f1")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy":acc.compute(predictions=predictions, references=labels),
            "f1-macro":macrof1.compute(predictions=predictions, references=labels, 
                                       average="macro")}

In [ ]:
def tokenize_function(examples):
    return bio_clinicalbert_tokenizer(examples['Assessment'], examples['Plan Subsection'],
                                      truncation="longest_first",
                                      max_length=512,
                                      verbose=True)
                    


In [ ]:
dataset = dataset.map(tokenize_function, batched=True)

In [ ]:
bio_clinicalbert_tokenizer.decode(dataset['valid'][52]['input_ids'])

In [ ]:
dataset['train'].set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataset['valid'].set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

In [18]:
data_collator = DataCollatorWithPadding(bio_clinicalbert_tokenizer,
                                        max_length=512, 
                                        padding="max_length",
                                        return_tensors="pt" )

In [19]:
trainer = Trainer(
    model=bio_clinicalbert_model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['valid'],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

In [20]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Assessment, ROW ID, HADM ID, Plan Subsection.
/gpfs/ysm/project/rtaylor/vs428/conda_envs/myenv/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4633
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1740


Epoch,Training Loss,Validation Loss,Accuracy,F1-macro
1,0.715300,0.597065,{'accuracy': 0.7487437185929648},{'f1': 0.766732642338792}
2,0.457600,0.672382,{'accuracy': 0.7805695142378559},{'f1': 0.7940059075142675}
3,0.301700,0.918875,{'accuracy': 0.7671691792294807},{'f1': 0.7860078751767712}


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Assessment, ROW ID, HADM ID, Plan Subsection.
***** Running Evaluation *****
  Num examples = 597
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Assessment, ROW ID, HADM ID, Plan Subsection.
***** Running Evaluation *****
  Num examples = 597
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved in test_trainer/checkpo

TrainOutput(global_step=1740, training_loss=0.45485844447694973, metrics={'train_runtime': 821.007, 'train_samples_per_second': 16.929, 'train_steps_per_second': 2.119, 'total_flos': 3657046227554304.0, 'train_loss': 0.45485844447694973, 'epoch': 3.0})